In [1]:
# Import libraries
import os
import sys
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set Pandas row limit
pd.set_option('display.max_rows', 250)

# Load the data from the CSV file
price_data = pd.read_csv('./data/AAPL_historical_data.csv')

In [2]:
# Add a "Change in price" column
price_data['change_in_price'] = price_data['c'].diff()
price_data.head()

,datetime,o,h,l,c,v,readable_time,date,change_in_price
0,1571385600000,235.36,235.36,235.05,235.05,268,2019-10-18 03:00:00,2019-10-18,NaN
1,1571386800000,234.91,234.91,234.91,234.91,179,2019-10-18 03:20:00,2019-10-18,-0.14
2,1571387700000,234.91,234.91,234.91,234.91,166,2019-10-18 03:35:00,2019-10-18,0.00
3,1571393700000,235.00,235.00,235.00,235.00,150,2019-10-18 05:15:00,2019-10-18,0.09
4,1571395200000,235.25,235.25,235.00,235.00,600,2019-10-18 05:40:00,2019-10-18,0.00


In [3]:
# Calculate 20 EMA (short term)
ema_20 = price_data['c'].transform(lambda x: x.ewm(span = 20).mean())
price_data['EMA_20'] = ema_20

# Calculate 200 EMA (long term)
ema_200 = price_data['c'].transform(lambda x: x.ewm(span = 200).mean())
price_data['EMA_200'] = ema_200

#  Calculate MACD 
ema_26 = price_data['c'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = price_data['c'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

ema_9_macd = macd.ewm(span = 9).mean()

# Calculate the VWAP
def vol_weighted_avg_price(date_group):
    vwap_col = []
    volume = date_group['v']
    high = date_group['h']
    low = date_group['l']
    
    # Calculate VWAP
    vwap_col = (volume * ((high + low)/2)).cumsum() / volume.cumsum()
    return pd.Series(vwap_col, index = date_group.index)
    
vwap = price_data.groupby('date').apply(vol_weighted_avg_price)
price_data['VWAP_50'] = vwap.reset_index(level=0, drop=True)

# Store the data in the data fram
price_data['MACD'] = macd
price_data['MACD_EMA'] = ema_9_macd
price_data['MACD_Diff'] = macd - ema_9_macd

price_data.tail(100)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff
31622,1593702300000,366.4401,366.9000,366.4300,366.5251,188871,2020-07-02 10:05:00,2020-07-02,0.0351,367.386398,365.606151,368.299449,-0.153132,0.083585,-0.236717
31623,1593702600000,366.5300,366.9099,365.9300,366.5700,189560,2020-07-02 10:10:00,2020-07-02,0.0449,367.308645,365.615742,368.265256,-0.193903,0.028088,-0.221991
31624,1593702900000,366.5900,367.0399,366.1460,366.9334,221625,2020-07-02 10:15:00,2020-07-02,0.3634,367.272908,365.628853,368.230427,-0.194648,-0.016459,-0.178188
31625,1593703200000,366.9500,367.0700,366.5100,366.8200,192303,2020-07-02 10:20:00,2020-07-02,-0.1134,367.229774,365.640705,368.204859,-0.202059,-0.053579,-0.148480
31626,1593703500000,366.8600,366.8600,365.9600,366.3550,194294,2020-07-02 10:25:00,2020-07-02,-0.4650,367.146462,365.647812,368.173237,-0.242657,-0.091395,-0.151262
31627,1593703800000,366.4000,366.7000,366.1601,366.4900,236656,2020-07-02 10:30:00,2020-07-02,0.1350,367.083942,365.656192,368.136614,-0.260930,-0.125302,-0.135628
31628,1593704100000,366.4600,366.6000,366.1500,366.1800,166249,2020-07-02 10:35:00,2020-07-02,-0.3100,366.997852,365.661404,368.110994,-0.297002,-0.159642,-0.137360
31629,1593704400000,366.1800,366.6500,365.8100,366.6123,224824,2020-07-02 10:40:00,2020-07-02,0.4323,366.961133,365.670866,368.074711,-0.287394,-0.185192,-0.102201
31630,1593704700000,366.6400,367.0100,366.4700,366.8600,138386,2020-07-02 10:45:00,2020-07-02,0.2477,366.951501,365.682698,368.059051,-0.256831,-0.199520,-0.057311
31631,1593705000000,366.8900,367.1800,366.7900,367.1045,153344,2020-07-02 10:50:00,2020-07-02,0.2445,366.966072,365.696846,368.045265,-0.210455,-0.201707,-0.008748


In [4]:
# Indicate buy points
def determine_buy_signal(df):
    buy_signals = []
    open_price = df['o']
    close = df['c']
    low = df['l']
    macd_diff = df['MACD_Diff']
    ema_20 = df['EMA_20']
    ema_200 = df['EMA_200']
    vwap = df['VWAP_50']
    

    for op, close, low, macd, ema, ema_long, vwap in zip(open_price, close, low, macd_diff, ema_20, ema_200, vwap):
        
        # Did the candle stick open above EMA
        if (op > ema and op > ema_long and 
            low > ema and low > ema_long and 
            low > vwap and 
            macd > 0.025 and macd < 0.05 and 
            close - op > 0.07):
            buy_signals.append('BUY')
        else:
            buy_signals.append('pass')
            continue
    
#         # Is the MACD gapping but not extended
#         if (macd > 0.02 and macd < 0.04):
#             pass
#         else:
#             buy_signals.append('pass')
#             continue
        
#         # MACD extended. Likely a reveral is coming
#         if (macd > 0.04):
#             buy_signals.append('pass')
#             continue
            
#         # Is the price near the 75% VWAP? Don't buy
            
#         # Is the price near the 50% VWAP? Buy
#         if (close > vwap):
#             pass
#         else:
#             buy_signals.append('pass')
#             continue
            
#         buy_signals.append('BUY')
    
    return pd.Series(buy_signals)

buy_column = determine_buy_signal(price_data)

price_data['Buy_Signals'] = buy_column

price_data.tail(50)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff,Buy_Signals
31672,1593717300000,366.3999,366.6400,366.2616,366.4200,162941,2020-07-02 14:15:00,2020-07-02,0.0400,366.762157,366.128589,367.726327,-0.110006,-0.078070,-0.031936,pass
31673,1593717600000,366.4100,366.7600,366.4100,366.7300,116627,2020-07-02 14:20:00,2020-07-02,0.3100,366.759094,366.134573,367.718765,-0.099031,-0.082262,-0.016769,pass
31674,1593717900000,366.7300,366.7699,366.4100,366.5698,151732,2020-07-02 14:25:00,2020-07-02,-0.1602,366.741066,366.138904,367.709118,-0.102083,-0.086226,-0.015857,pass
31675,1593718200000,366.5900,367.0000,366.2100,366.8400,227712,2020-07-02 14:30:00,2020-07-02,0.2702,366.750488,366.145880,367.695136,-0.081757,-0.085332,0.003576,pass
31676,1593718500000,366.8100,366.9000,366.3800,366.4650,186170,2020-07-02 14:35:00,2020-07-02,-0.3750,366.723299,366.149055,367.684324,-0.094814,-0.087229,-0.007586,pass
31677,1593718800000,366.4600,366.4800,365.9300,366.1600,298848,2020-07-02 14:40:00,2020-07-02,-0.3050,366.669651,366.149164,367.660385,-0.128295,-0.095442,-0.032853,pass
31678,1593719100000,366.1600,366.2400,365.5500,365.6781,321645,2020-07-02 14:45:00,2020-07-02,-0.4819,366.575218,366.144477,367.630163,-0.191506,-0.114655,-0.076851,pass
31679,1593719400000,365.6500,365.6500,364.4100,364.4100,580991,2020-07-02 14:50:00,2020-07-02,-1.2681,366.369007,366.127218,367.552172,-0.340007,-0.159725,-0.180282,pass
31680,1593719700000,364.4400,365.0100,363.6400,364.1200,1151973,2020-07-02 14:55:00,2020-07-02,-0.2900,366.154816,366.107246,367.371018,-0.475613,-0.222903,-0.252710,pass
31681,1593720000000,364.1200,364.6900,364.1100,364.3500,2268534,2020-07-02 15:00:00,2020-07-02,0.2300,365.982928,366.089761,367.075286,-0.558089,-0.289940,-0.268149,pass


In [5]:
# Incidate sell points
def determine_sell_signal(df):
    sell_signals = []
    open_price = df['o']
    close = df['c']
    low = df['l']
    macd_diff = df['MACD_Diff']
    ema_20 = df['EMA_20']
    vwap = df['VWAP_50']

    for op, close, low, macd, ema, vwap in zip(open_price, close, low, macd_diff, ema_20, vwap):
        
        # Did the candle stick close below the EMA line
        if (low < ema):
            sell_signals.append('SELL')
            continue
            
        # Big candle drop
        if (op - close > 0.081):
            sell_signals.append('SELL')
            continue
            
        # MACD extended
        if (macd > 0.07):
            sell_signals.append('SELL')
            continue
        
        sell_signals.append('pass')
        
    return pd.Series(sell_signals)

sell_column = determine_sell_signal(price_data)

price_data['Sell_Signals'] = sell_column

price_data.tail(50)



,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff,Buy_Signals,Sell_Signals
31672,1593717300000,366.3999,366.6400,366.2616,366.4200,162941,2020-07-02 14:15:00,2020-07-02,0.0400,366.762157,366.128589,367.726327,-0.110006,-0.078070,-0.031936,pass,SELL
31673,1593717600000,366.4100,366.7600,366.4100,366.7300,116627,2020-07-02 14:20:00,2020-07-02,0.3100,366.759094,366.134573,367.718765,-0.099031,-0.082262,-0.016769,pass,SELL
31674,1593717900000,366.7300,366.7699,366.4100,366.5698,151732,2020-07-02 14:25:00,2020-07-02,-0.1602,366.741066,366.138904,367.709118,-0.102083,-0.086226,-0.015857,pass,SELL
31675,1593718200000,366.5900,367.0000,366.2100,366.8400,227712,2020-07-02 14:30:00,2020-07-02,0.2702,366.750488,366.145880,367.695136,-0.081757,-0.085332,0.003576,pass,SELL
31676,1593718500000,366.8100,366.9000,366.3800,366.4650,186170,2020-07-02 14:35:00,2020-07-02,-0.3750,366.723299,366.149055,367.684324,-0.094814,-0.087229,-0.007586,pass,SELL
31677,1593718800000,366.4600,366.4800,365.9300,366.1600,298848,2020-07-02 14:40:00,2020-07-02,-0.3050,366.669651,366.149164,367.660385,-0.128295,-0.095442,-0.032853,pass,SELL
31678,1593719100000,366.1600,366.2400,365.5500,365.6781,321645,2020-07-02 14:45:00,2020-07-02,-0.4819,366.575218,366.144477,367.630163,-0.191506,-0.114655,-0.076851,pass,SELL
31679,1593719400000,365.6500,365.6500,364.4100,364.4100,580991,2020-07-02 14:50:00,2020-07-02,-1.2681,366.369007,366.127218,367.552172,-0.340007,-0.159725,-0.180282,pass,SELL
31680,1593719700000,364.4400,365.0100,363.6400,364.1200,1151973,2020-07-02 14:55:00,2020-07-02,-0.2900,366.154816,366.107246,367.371018,-0.475613,-0.222903,-0.252710,pass,SELL
31681,1593720000000,364.1200,364.6900,364.1100,364.3500,2268534,2020-07-02 15:00:00,2020-07-02,0.2300,365.982928,366.089761,367.075286,-0.558089,-0.289940,-0.268149,pass,SELL


In [6]:
# Calculate enter and exit points
# This will strictly use the buy/sell signal. As soon as a buy is reached, we buy. After that, we hold until we get a sell signal

def enter_exit_points(df):
    enter_exit_points = []
    buy_signals = df['Buy_Signals']
    sell_signals = df['Sell_Signals']
    
    holding = False
    
    for bs, ss in zip(buy_signals, sell_signals):
        
        if(not holding):
            if (bs == 'BUY'):
                enter_exit_points.append('ENTER')
                holding = True
                continue
            else:
                enter_exit_points.append('wait')
                continue
        else:
            if(ss == 'SELL'):
                enter_exit_points.append('EXIT')
                holding = False
                continue
            else:
                enter_exit_points.append('hold')
                continue
                
    return pd.Series(enter_exit_points)

enter_exit_column = enter_exit_points(price_data)

price_data['Enter_Exit'] = enter_exit_column

price_data.tail(50)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff,Buy_Signals,Sell_Signals,Enter_Exit
31672,1593717300000,366.3999,366.6400,366.2616,366.4200,162941,2020-07-02 14:15:00,2020-07-02,0.0400,366.762157,366.128589,367.726327,-0.110006,-0.078070,-0.031936,pass,SELL,wait
31673,1593717600000,366.4100,366.7600,366.4100,366.7300,116627,2020-07-02 14:20:00,2020-07-02,0.3100,366.759094,366.134573,367.718765,-0.099031,-0.082262,-0.016769,pass,SELL,wait
31674,1593717900000,366.7300,366.7699,366.4100,366.5698,151732,2020-07-02 14:25:00,2020-07-02,-0.1602,366.741066,366.138904,367.709118,-0.102083,-0.086226,-0.015857,pass,SELL,wait
31675,1593718200000,366.5900,367.0000,366.2100,366.8400,227712,2020-07-02 14:30:00,2020-07-02,0.2702,366.750488,366.145880,367.695136,-0.081757,-0.085332,0.003576,pass,SELL,wait
31676,1593718500000,366.8100,366.9000,366.3800,366.4650,186170,2020-07-02 14:35:00,2020-07-02,-0.3750,366.723299,366.149055,367.684324,-0.094814,-0.087229,-0.007586,pass,SELL,wait
31677,1593718800000,366.4600,366.4800,365.9300,366.1600,298848,2020-07-02 14:40:00,2020-07-02,-0.3050,366.669651,366.149164,367.660385,-0.128295,-0.095442,-0.032853,pass,SELL,wait
31678,1593719100000,366.1600,366.2400,365.5500,365.6781,321645,2020-07-02 14:45:00,2020-07-02,-0.4819,366.575218,366.144477,367.630163,-0.191506,-0.114655,-0.076851,pass,SELL,wait
31679,1593719400000,365.6500,365.6500,364.4100,364.4100,580991,2020-07-02 14:50:00,2020-07-02,-1.2681,366.369007,366.127218,367.552172,-0.340007,-0.159725,-0.180282,pass,SELL,wait
31680,1593719700000,364.4400,365.0100,363.6400,364.1200,1151973,2020-07-02 14:55:00,2020-07-02,-0.2900,366.154816,366.107246,367.371018,-0.475613,-0.222903,-0.252710,pass,SELL,wait
31681,1593720000000,364.1200,364.6900,364.1100,364.3500,2268534,2020-07-02 15:00:00,2020-07-02,0.2300,365.982928,366.089761,367.075286,-0.558089,-0.289940,-0.268149,pass,SELL,wait


In [ ]:
# Add an account balance total, starting with $1000
def track_account_balance(df):
    account_balance = []
    current_balance = 150000.0
    enter_exit = df['Enter_Exit']
    close = df['c']
    
    holding = False
    account_balance.append(150000.0)
    
    for ee, close in zip(enter_exit, close):
        if(ee == 'ENTER' and holding == False):
            current_balance -= (close * 30.0)
            account_balance.append(current_balance)
            holding = True
            continue
        
        if(ee == 'EXIT' and holding == True):
            current_balance += (close * 30.0)
            account_balance.append(current_balance)
            holding = False
            continue
        
        account_balance.append(current_balance)
            
    return pd.Series(account_balance)

account_balance_col = track_account_balance(price_data)

price_data['Account_Balance'] = account_balance_col
price_data.tail(400)

# Export to CSV
price_data.to_csv("result.csv")
price_data.tail()